In [46]:
import re
import pandas as pd
import nltk
from nltk.tokenize import WhitespaceTokenizer
import pandas as pd

In [47]:
tk = WhitespaceTokenizer()

RE_SPACES = re.compile("\s+")
RE_HASHTAG = re.compile("[@#][_a-z0-9]+")
RE_EMOTICONS = re.compile("(:-?\))|(:p)|(:d+)|(:-?\()|(:/)|(;-?\))|(<3)|(=\))|(\)-?:)|(:'\()|(8\))")
RE_HTTP = re.compile("http(s)?://[/\.a-z0-9]+")

In [48]:
tweets = pd.read_csv('train.csv', header=0, sep=',').iloc[: , -1]
tweets.head(6)

0                                        Not Available
1    IOS 9 App Transport Security. Mm need to check...
2    Mar if you have an iOS device, you should down...
3    @jimmie_vanagon my phone does not run on lates...
4    Not sure how to start your publication on iOS?...
5    Two Dollar Tuesday is here with Forklift 2, Qu...
Name: Tweet, dtype: object

In [50]:
sentiments = pd.read_csv('train.csv', header=0, sep=',').iloc[: , -2]
sentiments.head(6)

0    negative
1     neutral
2     neutral
3    negative
4    positive
5     neutral
Name: Category, dtype: object

In [83]:
class Tokenizer():
    @staticmethod
    def tokenize(tweets, sentiments):
        zero_entry = {"sentiment": [''], "tweet": ['']}
        normalized_tweets = pd.DataFrame(zero_entry)
        for (i, j) in zip(tweets.index, sentiments.index):
            tweet = BeforeTokenizationNormalizer.normalize(tweets.iat[i])
            tweet = tk.tokenize(tweet)
            new_row = pd.DataFrame({"sentiment": [sentiments.iat[j]], "tweet": [tweet]})
            normalized_tweets = pd.concat([normalized_tweets, new_row], ignore_index = True)

        normalized_tweets = normalized_tweets.iloc[1:]
        return normalized_tweets
    
class BeforeTokenizationNormalizer():
    @staticmethod
    def normalize(text):
        text = text.strip().lower()
        text = text.replace('&nbsp;', ' ')
        text = text.replace('&lt;', '<')
        text = text.replace('&gt;', '>')
        text = text.replace('&amp;', '&')
        text = text.replace('&pound;', u'£')
        text = text.replace('&euro;', u'€')
        text = text.replace('&copy;', u'©')
        text = text.replace('&reg;', u'®')
        return text

In [84]:
print(Tokenizer.tokenize(tweets, sentiments))

     sentiment                                              tweet
1     negative                                   [not, available]
2      neutral  [ios, 9, app, transport, security., mm, need, ...
3      neutral  [mar, if, you, have, an, ios, device,, you, sh...
4     negative  [@jimmie_vanagon, my, phone, does, not, run, o...
5     positive  [not, sure, how, to, start, your, publication,...
...        ...                                                ...
5966   neutral  [@youaremyarsenal, wouldn't, surprise, me, if,...
5967   neutral  [rib, injury, for, zlatan, against, russia, is...
5968   neutral  [noooooo!, i, was, hoping, to, see, zlatan, be...
5969   neutral                                   [not, available]
5970   neutral                                   [not, available]

[5970 rows x 2 columns]


# Klasyfikator

In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import re
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report

In [2]:
tweets_set= pd.read_csv("train.csv", sep=',', header = 1)

classes = ['negative', 'positive', 'neutral']
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

temp = np.random.rand(len(tweets_set)) < 0.80
tweetsTrain = tweets_set[temp]
tweetsTest = tweets_set[~temp]

tweetsTrainData = []
tweetsTrainLabel = []

tweetsTestData = []
tweetsTestLabel = []

In [3]:
for index, tweet in tweetsTrain.iterrows():
    tweet[2] = re.sub(r'@\w+', '', tweet[2])
    querywords = tweet[2].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[2] = ' '.join(resultwords)
    tweetsTrainData.append(tweet[2])
    tweetsTrainLabel.append(tweet[1])

for index, tweet in tweetsTest.iterrows():
    tweet[2] = re.sub(r'@\w+', '', tweet[2])
    querywords = tweet[2].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[2] = ' '.join(resultwords)
    tweetsTestData.append(tweet[2])
    tweetsTestLabel.append(tweet[1])

In [4]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.80,
                             sublinear_tf=True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(tweetsTrainData)
test_vectors = vectorizer.transform(tweetsTestData)

In [5]:
classifierNB = MultinomialNB()
classifierNB.fit(train_vectors, tweetsTrainLabel)
predictionNB=classifierNB.predict(test_vectors)

In [6]:
print("Results for MultinomialNB()")
print(classification_report(tweetsTestLabel, predictionNB))

Results for MultinomialNB()
              precision    recall  f1-score   support

    negative       0.56      0.07      0.13       198
     neutral       0.47      0.37      0.41       451
    positive       0.57      0.84      0.68       577

    accuracy                           0.54      1226
   macro avg       0.53      0.43      0.41      1226
weighted avg       0.53      0.54      0.49      1226



In [16]:
import os
import pandas as pd
import numpy as np
import re
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import csv

tweetsTest = pd.read_csv("test.csv", sep=',', header = 1)
tweetsTrain = pd.read_csv("train.csv", sep=',', header = 1)

classes = ['negative', 'positive', 'neutral']
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

tweetsTrainData = []
tweetsTrainLabel = []

tweetsTestId = []
tweetsTestData = []
tweetsTestLabel = []

for index, tweet in tweetsTrain.iterrows():
    tweet[2] = re.sub(r'@\w+', '', tweet[2])
    querywords = tweet[2].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[2] = ' '.join(resultwords)
    tweetsTrainData.append(tweet[2])
    tweetsTrainLabel.append(tweet[1])

for index, tweet in tweetsTest.iterrows():
    if str(tweet[1]) != "nan":
        tweet[1] = re.sub(r'@\w+', '', tweet[1])
        querywords = tweet[1].split()
        resultwords  = [word for word in querywords if word.lower() not in stop_words]
        tweet[1] = ' '.join(resultwords)
        tweetsTestData.append(tweet[1])
        tweetsTestId.append(tweet[0])
    
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.80,
                             sublinear_tf=True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(tweetsTrainData)
test_vectors = vectorizer.transform(tweetsTestData)

classifierNB = MultinomialNB()
classifierNB.fit(train_vectors, tweetsTrainLabel)
predictionNB=classifierNB.predict(test_vectors)

header = {'Id': tweetsTestId, 'Category': predictionNB}
result = pd.DataFrame(data = header)
result.to_csv('result.csv', sep = ',', columns=['Id', 'Category'], index=False)